# Final project

## Project title

Research Question: Which UvA courses do I meet the entry requirements for? XXX






## Process Book

1. Don’t forget the process book. Maintain it every day you’ve worked on the project. This is the main way for 
   us to see what you’ve learned and is an important part of the evaluation.
2. Don’t use it for everything. Don’t spend too much time on your process book! Ten minutes 
   every day you’ve worked on the project on this, should be more than enough.
3. Keep it simple and short. Use simple language, short sentences, and be as straightforward as possible when 
   documenting your learning process.
4. Create a simple template. In order to organize your process book, it is nice to create and use a template:


### Date of entry: 25. Jan
#### What I’ve worked on: 
Git
- Lecture git
- Tried out git by myself
- Created git repository

Project choice
- Learn to use tableau and uber data as I am applying to intern at uber and tableau is a desired skill

Tableau
- Downloaded tableau 
- See what I can use tableau for to decide which project I want to do specifically
- Learn tableau basics
- Experiment with DP datasets (e.g., weather data) in tableau
- Experiment with uber selection dataset --> too easy (?)
    
#### What problems I encountered:
- Cannot use dataset I first thought about using

#### What I learned:
- How to use git
- Tableau basics https://www.youtube.com/watch?v=jEgVto5QME8
- I want to gather my own uber data e.g., through web-scraping, or API

#### Which resources did I use:
- Course material, YouTube, tableau, tableau user guide, uber data XXX




### Date of entry: 26. Jan
#### What I’ve worked on: 
Tableau
- Learned more functions of tableau and experimented with these  

Project
- Created jupyter notebook
- Started process book

Brainstorm
- Potentially make website with tableau dashboard
- Which data can I use? 

Find data
- Look for uber APIs

#### What problems I encountered: 
- Uber APIs are not open, I can not use them 

#### What I learned: 
Tableau
- Basic functions: https://www.youtube.com/watch?v=yejulE4b-3Y
- Use and join (csv) files: https://www.youtube.com/watch?v=5WXrJ1JvTOo
- Integrate tableau dashboard in website: https://www.youtube.com/watch?v=wJ2CHIJalNU

#### Which resources did I use:
- Course material, YouTube, tableau, tableau user guide



### Date of entry: 27. Jan
#### What I’ve worked on: 
- Find new project ideas
- Decided for final topic/Question
- Changed strategy for scraping data: work with individual course pages instead of course overview

#### What problems I encountered: 
- Url of studiegids does not change per page of courses

#### What I learned: 

#### Which resources did I use:
- Multiple forms to try to solve the url problem, but didn't find a solution




### Date of entry: 30. Jan
#### What I’ve worked on: 

#### What problems I encountered: 

#### What I learned: 

#### Which resources did I use:

## To-Do

1. Scrape / Transform data 
2. Import data into tableau
3. (Webpage with tableau dashboard)

## 1. Data

Relevant personal data

Programme: ""
Level: Bachelor's, Master's (Master's, Master), Exchange programme ('Exchange Programme Exchange Programme UvA', 'Exchange Programme Exchange Programme'), pre-Master's programme, Secondary Subject/Bijvak('?'), 'Honours MSc', 'Research Master's
Year of the studies: ""
Credits obtained this/1st year: ""

url: https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course


Relevant course information 

Course name: Course name
Entry requirements: Course > Entry requirements > Split by sentence & look for keywords (if not present, add to dict with unknown entry requirements)
Level: Course > Is part of > loop through Level list (s.a.) 
Field: Course > Is part of > remove Level > everything left 
(Coordinator name: Course > Lecturer(s) > Coordinator ('co-ordinator')
(Coordinator email: Course > Lecturer(s) > Coordinator (crawl) > Email)


In [2]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing

def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None
    """
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'
        }
        with closing(get(url, stream=True, headers=headers)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None
    except RequestException as e:
        print('The following error occurred during HTTP GET request to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns true if the response seems to be HTML, false otherwise
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200
            and content_type is not None
            and content_type.find('html') > -1)

In [10]:
from bs4 import BeautifulSoup
import re
import pandas as pd
import requests
import numpy as np
from math import ceil
import sys


url = 'https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course'
html = simple_get(url)
dom = BeautifulSoup(html, 'html.parser')

course_names = []

all_names = dom.find_all('tr', class_ = 'result') #only course names

for name in all_names:
    course_names.append(one_entry.find('a', class_ = 'icon-arrow').text)


print(all_names)
    
#all_slideouts = 


#def extract_courses: 
print(html)

#ERROR: cannot access the correct html with this url

[]
b'<!DOCTYPE html>\n<!--[if lt IE 8]><html class="no-js ie7"><![endif]--><!--[if IE 8]><html class="no-js ie8"><![endif]--><!--[if gt IE 8]><!--><html xml:lang="en" lang="en"><!--<![endif]--><head><script type="text/javascript" src="//static.queue-it.net/script/queueclient.min.js"> </script><script data-queueit-c="sans" type="text/javascript" src="//static.queue-it.net/script/queueconfigloader.min.js"> </script><meta name="viewport" content="width=device-width"/><meta http-equiv="X-UA-Compatible" content="IE=Edge,chrome=1"/><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"/><title>Course | UvA Studiegids 2022 - 2023</title><link href="/xmlpages/resources/TXP/uva/studiegidswebsite/img/favicon.png" rel="icon" type="image/png"/><link href="/xmlpages/resources/TXP/uva/studiegidswebsite/css/style.css" rel="stylesheet" type="text/css"/><link href="/xmlpages/resources/TXP/uva/studiegidswebsite/css/mobile.css" rel="stylesheet" type="text/css"/><!--[if IE]><link rel="styleshe